In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

set_log_level("ERROR")

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')
df

,ds,y
3692,2012-05-01,211
3691,2012-05-02,201
3690,2012-05-03,204
3689,2012-05-04,210
3688,2012-05-05,180
...,...,...
4,2022-06-07,276
3,2022-06-08,278
2,2022-06-09,284
1,2022-06-10,253


In [5]:
len(df)

3693

In [4]:
m = NeuralProphet(
  yearly_seasonality=True,
  weekly_seasonality=True,
  daily_seasonality=True,
  n_lags=4*7+1,
  n_forecasts=8,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
).add_country_holidays("CA")

metrics = m.fit(df, 
                freq='D', 
                # progress='plot'
                )
print(metrics.tail(1))


WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

     SmoothL1Loss        MAE       RMSE  RegLoss
133      0.003354  12.848008  16.585629      0.0


In [14]:
future = m.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False
future

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:2311: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(future_df)



,ds,y
0,2022-05-14,217
1,2022-05-15,247
2,2022-05-16,301
3,2022-05-17,271
4,2022-05-18,264
5,2022-05-19,250
6,2022-05-20,262
7,2022-05-21,173
8,2022-05-22,218
9,2022-05-23,242


In [15]:
forecast = m.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','inflow','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='D')
forecast_output['inflow'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



,ds,inflow,timestamp
0,2022-06-12,210.330292,2022-06-13 01:37:29
1,2022-06-13,282.891418,2022-06-13 01:37:29
2,2022-06-14,254.657578,2022-06-13 01:37:29
3,2022-06-15,249.184326,2022-06-13 01:37:29
4,2022-06-16,248.595032,2022-06-13 01:37:29
5,2022-06-17,255.053741,2022-06-13 01:37:29
6,2022-06-18,217.779648,2022-06-13 01:37:29
7,2022-06-19,208.103027,2022-06-13 01:37:29
